# Monthly wages for females and males
Aggregate by:
* Total overall, whole world
* Continent

Change data to ratio of women / men wages

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [2]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

In [3]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [4]:
# import US specific data (median dollar earnings per month)
us_data = pd.read_csv("median_US_earnings.csv")
us_data.head()  

,year,female,male
0,1995,3133,4386
1,1996,3216,4360
2,1997,3316,4471
3,1998,3388,4630
4,1999,3376,4668


In [5]:
wages_fem = dfs[0][1].iloc[0:85, :]  # monthly wages for females
wages_m = dfs[1][1].iloc[0:85, :] # monthly wages for males
# wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
# wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows
wages_fem_orig = wages_fem.copy()
wages_m_orig =wages_m.copy()
# wages_fem = wages_fem[~wages_fem['2011'].isna()]
# wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries


(85, 18) (85, 18)


In [6]:
wages_fem.iloc[:, 1:] = wages_fem.iloc[:, 1:].fillna(axis=1, method='pad')
wages_m.iloc[:, 1:] = wages_m.iloc[:, 1:].fillna(axis=1, method='pad')
wages_fem.iloc[:, 1:] = wages_fem.iloc[:, 1:].fillna(axis=1, method='bfill')
wages_m.iloc[:, 1:] = wages_m.iloc[:, 1:].fillna(axis=1, method='bfill')

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [7]:
print(wages_fem.isna().sum().sum())
print(wages_m.isna().sum().sum())

0
0


In [8]:
# wages_fem_orig.head()
# wages_fem.head()

In [40]:
# process us data separartely
us_data2 = us_data.transpose()
us_data2.columns = us_data2.iloc[0,:]
print(us_data2.isna().sum().sum())
wages_fem_us = pd.DataFrame(us_data2.iloc[1,:])
wages_m_us = pd.DataFrame(us_data2.iloc[2,:])


0


In [41]:
wages_m_us.rename(columns={'male': 'value_m'}, inplace=True)
wages_fem_us.rename(columns={'female': 'value_f'}, inplace=True)


In [44]:
# wages_fem_us['gender'] = 'female'
wages_fem_us.reset_index(inplace=True)
# wages_m_us['gender'] = 'male'
wages_m_us.reset_index(inplace=True)

In [47]:
us_all = pd.merge(wages_m_us, wages_fem_us, on='year')


,year,value_m,value_f
0,1995,4386,3133
1,1996,4360,3216
2,1997,4471,3316
3,1998,4630,3388
4,1999,4668,3376


In [48]:
us_all['w_m_ratio'] = us_all['value_f'] / us_all['value_m']

In [50]:
us_all['ISO'] = 'USA'
us_all.head()

,year,value_m,value_f,w_m_ratio,ISO
0,1995,4386,3133,0.714318,USA
1,1996,4360,3216,0.737615,USA
2,1997,4471,3316,0.741669,USA
3,1998,4630,3388,0.731749,USA
4,1999,4668,3376,0.723222,USA


In [51]:
# # wages_us2.drop(labels='index', axis=1, inplace=True)
us_all.drop(labels=['value_m', 'value_f'], axis=1, inplace=True)

In [52]:
us_all.head()

,year,w_m_ratio,ISO
0,1995,0.714318,USA
1,1996,0.737615,USA
2,1997,0.741669,USA
3,1998,0.731749,USA
4,1999,0.723222,USA


In [53]:
## combine BOTH genders one dataset for ONE country
""" 
1. Narrow down country
2. Separately for each M / F dataset, melt the dataset
3. concat the melted datasets together
"""
all_countries = pd.DataFrame()
countries = list(wages_fem['ISO'].unique())
countries.remove("MLT")

## Choose a country
for c in countries:
    f1 = wages_fem[wages_fem['ISO'] == c]
    f2 = f1.melt().iloc[1:, :]
    f2.columns = ['year', 'value_f']
    m1 = wages_m[wages_m['ISO'] == c]
    m2 = m1.melt().iloc[1:, :]
    m2.columns = ['year', 'value_m']
    all = pd.merge(f2, m2, on='year')
    all['w_m_ratio'] = all['value_f'] / all['value_m']
    all['ISO'] = c
    all.drop(labels=['value_f', 'value_m'], axis=1, inplace=True)
    all_countries = pd.concat([all_countries, all], axis=0)


In [54]:
# all_countries.head()
print(all_countries.shape)

(1428, 3)


In [56]:
all_countries.head()
# all_countries.tail()

,year,w_m_ratio,ISO
0,1995,0.728161,ARG
1,1996,0.728161,ARG
2,1997,0.746303,ARG
3,1998,0.732799,ARG
4,1999,0.76014,ARG


In [58]:
all_countries2 = pd.concat([all_countries, us_all], axis=0)
all_countries2.shape

(1459, 3)

In [59]:
df = all_countries2
df.dropna(axis=0, inplace=True)
df.shape

(1459, 3)

In [60]:
df.head()

,year,w_m_ratio,ISO
0,1995,0.728161,ARG
1,1996,0.728161,ARG
2,1997,0.746303,ARG
3,1998,0.732799,ARG
4,1999,0.76014,ARG


In [61]:
## Map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
country_names_dict = { v: k for k, v in country_names.items()}
def convert_iso_to_name(iso):
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso
# df['country'] = df['ISO']
df['country'] = df['ISO'].apply(convert_iso_to_name)
# df.drop(labels='ISO', axis=1, inplace=True)

In [64]:
df['w_m_ratio'] = df['w_m_ratio'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 30
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       1459 non-null   object 
 1   w_m_ratio  1459 non-null   float64
 2   ISO        1459 non-null   object 
 3   country    1459 non-null   object 
dtypes: float64(1), object(3)
memory usage: 57.0+ KB


In [22]:
# df = df[['year', 'country', 'value', 'gender']]

In [65]:
df.head()

,year,w_m_ratio,ISO,country
0,1995,0.728161,ARG,Argentina
1,1996,0.728161,ARG,Argentina
2,1997,0.746303,ARG,Argentina
3,1998,0.732799,ARG,Argentina
4,1999,0.760140,ARG,Argentina


In [67]:
# df['country'].nunique() # 85 countries

85

In [68]:
## TO EXPORT AS CSV
df.to_csv("wages_ratio.csv", index=False)

In [71]:
## Aggregate for all years
df_agg = df.groupby(by='year').mean()


In [74]:
df_agg.reset_index(inplace=True)
df_agg.head()

,year,w_m_ratio
0,1995,0.714318
1,1996,0.737615
2,1997,0.741669
3,1998,0.731749
4,1999,0.723222


In [72]:
df_agg.to_csv("wages_agg.csv", index=False)

In [ ]:
# TODO: organize by continent? 
# Need to label countries by continent